### 1. `mlflow.sklearn.autolog()`

#### What it does:

This function enables **automatic logging** of all relevant information about the training run. Once called, MLflow will:

* Log all parameters passed to the model (e.g., `C`, `max_iter`)
* Log metrics like accuracy, loss, etc.
* Save the trained model to the run’s artifact location
* Track the structure of the model and training time

#### Why it matters:

Without autologging, you would have to explicitly call:

```python
mlflow.log_param("C", C)
mlflow.log_metric("accuracy", acc)
mlflow.sklearn.log_model(model, "model")
```

With `mlflow.sklearn.autolog()`, this is done **automatically**.

#### Benefit:

* Saves time
* Reduces boilerplate code
* Makes experiment tracking more consistent

---

### 2. `StandardScaler`

```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
```

#### What it does:

`StandardScaler` standardizes the dataset's features by removing the mean and scaling to unit variance:

* Mean becomes 0
* Standard deviation becomes 1


In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Enable autologging
mlflow.sklearn.autolog()

# Load and preprocess dataset
df = pd.read_csv("bankdata.csv")

# Encode categorical variables
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop("y", axis=1)
y = df["y"]

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model inside MLflow run
C = 0.1
with mlflow.start_run():
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)


    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"Accuracy: {acc:.4f}")